In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 1

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [37]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=False)

Attack dataset was already established previosly, loading dataset from "pickle/tinyimagenet/mobilenetv2/attack_train.p".


In [38]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 1.5733,  1.0517,  0.9332]],  [[ 1.4803,  1.2408,  1.0936]],  [[ 1.8819,  1.4202,  0.9920]],  [[ 1.3752,  0.8301,  0.7765]],  [[ 0.6191,  0.4609,  0.4162]],  [[ 1.7589,  1.3028,  1.0077]],  [[ 1.7512,  1.6551,  1.4562]],  [[ 3.0344,  2.4331,  2.1334]],  [[ 0.8850,  0.8362,  0.6184]],  [[ 2.1119,  1.4899,  1.1929]],  [[ 1.5477,  0.9689,  0.3738]],  [[ 2.4597,  1.9625,  1.8480]],  [[ 2.1375,  1.5997,  1.3010]],  [[ 3.3657,  3.1185,  2.7974]],  [[ 3.3030,  2.5985,  1.6694]],  [[ 2.5688,  0.8300,  0.7213]],  [[ 4.4556,  3.3841,  3.3115]],  [[ 2.3506,  1.3098,  1.1554]],  [[ 1.4182,  0.6729,  0.2420]],  [[ 2.2486,  2.0390,  1.8939]],  [[ 2.3650,  2.2876,  1.8899]],  [[ 3.4936,  2.9939,  2.9636]],  [[ 3.8788,  3.2878,  3.0938]],  [[ 1.9715,  1.2318,  0.8861]],  [[ 2.2502,  1.8399,  1.8250]],  [[ 2.4132,  1.8050,  1.7030]],  [[ 1.5798,  1.4761,  1.2136]],  [[ 1.2507,  0.5630,  0.5377]],  [[ 1.3365,  1.0962,  0.8819]],  [[ 1.6777,  1.5569,  1.3446]],  [[ 3.2561,  3.2542,  2.0490]], 

### Attack Model

Simple Model

In [39]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [40]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [41]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [42]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE, test_dataset=False, standardize=False)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.9686, 0.9686, 0.9725,  ..., 0.9765, 0.9725, 0.9686], [0.9608, 0.9647, 0.9647,  ..., 0.9725, 0.9686, 0.9686], [0.9647, 0.9647, 0.9686,  ..., 0.9686, 0.9647, 0.9647], ..., [0.3843, 0.1294, 0.3294,  ..., 0.6627, 0.6980, 0.4980], [0.1333, 0.3804, 0.4118,  ..., 0.4078, 0.5020, 0.2902], [0.1569, 0.2863, 0.4353,  ..., 0.4510, 0.6549, 0.4078]],[[0.9765, 0.9765, 0.9804,  ..., 0.9843, 0.9804, 0.9765], [0.9686, 0.9725, 0.9725,  ..., 0.9804, 0.9765, 0.9765], [0.9725, 0.9725, 0.9765,  ..., 0.9765, 0.9725, 0.9725], ..., [0.3647, 0.1137, 0.3176,  ..., 0.5765, 0.6118, 0.4118], [0.1098, 0.3608, 0.4000,  ..., 0.3098, 0.4039, 0.1922], [0.1294, 0.2667, 0.4235,  ..., 0.3490, 0.5490, 0.3020]],[[0.9333, 0.9333, 0.9373,  ..., 0.9412, 0.9373, 0.9333], [0.9255, 0.9294, 0.9294,  ..., 0.9373, 0.9333, 0.9333], [0.9294, 0.9294, 0.9333,  ..., 0.9333, 0.9294, 0.9294], ..., [0.1451, 0.0000,

In [43]:
type(attack_eval_post_loader)

torch.utils.data.dataloader.DataLoader

Attack Evaluation DL Sample

In [44]:
eval_sample_iter = iter(attack_eval_post_loader)
#print(str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

### Train Attack model

In [45]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Attack model state dictionary already available, loading into input model from attack_models/attack_mobilenet_tinyimage.pth!


### Evaluation

In [46]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 32.92


32.92